In [1]:
import pathlib
import pandas as pd
import xml.etree.ElementTree as et

# define the path
currentDirectory = pathlib.Path('D:/TU/ED/EX2/CoE_dataset/Dev_Set/XML')

# define the pattern
currentPattern = "*.xml"
all_xml_files = []
for currentFile in currentDirectory.glob(currentPattern):
    all_xml_files.append(currentFile.parts[-1])
    
#print(all_xml_files)

In [2]:
def parse_XML(xml_file, name, df_cols):
            global out_df

            xtree = et.parse(xml_file)
            xroot = xtree.getroot()
            fin = []
            res = []
            res.append(name.split(".")[0])
            
            for node in xroot:
                for i in node_cols:
                    res.append(node.attrib.get(i))

            fin.append(res)
            print(fin)
            out_df = pd.DataFrame(data=fin, columns =meta_cols)
            return out_df
        
def parse_XML1(xml_file, name, df_cols, df):
            global out_df
            xtree = et.parse(xml_file)
            xroot = xtree.getroot()
            fin = []
            res = []
            if name.split(".")[0] == "Mr":
                res.append("Mr._Go")
            else:
                res.append(name.split(".")[0])
            for node in xroot:
                for i in node_cols:
                    res.append(node.attrib.get(i))

            fin.append(res)
            df1 = pd.DataFrame(data=fin, columns =meta_cols)
            
            frames = [df, df1]
            
            out_df = pd.concat(frames)
            
            return out_df

In [3]:
meta_cols = ["filename","language", "year", "genre", "country",
            "runtime", "rated"]

node_cols = ["language", "year", "genre", "country",
            "runtime", "rated"]

In [4]:
c = 1
for i in all_xml_files:
    if c==1:
        parse_XML("XML/" + i, i,  meta_cols)
        c=43
    else:
        parse_XML1("XML/"+i, i, meta_cols, out_df)

[['American_Gangster', 'English', '2007', 'Biography, Crime, Drama', 'USA, UK', '157 min', 'R']]


In [5]:
df_ground = pd.read_excel("dev_set_groundtruth_and_trailers.xls",index_col=False)

In [6]:
a = df_ground["filename"].sort_values()
b = out_df["filename"].sort_values()

for i in a.values:
    if i in b.values:
        continue
    else:
        print(i)

In [7]:
df_join = pd.merge(df_ground, out_df, on="filename", how="inner")

In [8]:
df_join = df_join.drop(["movie", "filename", "trailer"], axis=1)

In [9]:
import numpy as np

count=0
for i in df_join["genre"].values:
    df_join["genre"].values[count] = i.replace(" ", "")
    count+=1
    
count=0
for i in df_join["language"].values:
    df_join["language"].values[count] = i.replace(" ", "")
    count+=1
    
    
count=0
for i in df_join["country"].values:
    df_join["country"].values[count] = i.replace(" ", "")
    count+=1

In [10]:
genre = df_join.genre.str.split(",", expand=True).stack()
language = df_join.language.str.split(",", expand=True).stack()
country = df_join.country.str.split(",", expand=True).stack()

In [11]:
df_join = pd.concat([df_join, 
                pd.get_dummies(genre, prefix="g").groupby(level=0).sum()],axis=1) \
                .drop(["genre"],axis=1)

df_join = pd.concat([df_join, 
                pd.get_dummies(language, prefix="l").groupby(level=0).sum()],axis=1) \
                .drop(["language"],axis=1)

df_join = pd.concat([df_join, 
                pd.get_dummies(country, prefix="c").groupby(level=0).sum()],axis=1) \
                .drop(["country"],axis=1)

In [12]:
df_join.columns

Index(['goodforairplane', 'year', 'runtime', 'rated', 'g_Action',
       'g_Adventure', 'g_Animation', 'g_Biography', 'g_Comedy', 'g_Crime',
       'g_Drama', 'g_Family', 'g_Fantasy', 'g_Horror', 'g_Music', 'g_Musical',
       'g_Mystery', 'g_Romance', 'g_Sci-Fi', 'g_Sport', 'g_Thriller', 'g_War',
       'g_Western', 'l_AmericanSignLanguage', 'l_Arabic', 'l_Croatian',
       'l_Danish', 'l_Dutch', 'l_English', 'l_Filipino', 'l_French',
       'l_German', 'l_Greek', 'l_Hindi', 'l_Hungarian', 'l_Inuktitut',
       'l_Italian', 'l_Japanese', 'l_Korean', 'l_Latin', 'l_Mandarin',
       'l_Navajo', 'l_Russian', 'l_Serbian', 'l_Spanish', 'l_Swahili',
       'l_Urdu', 'l_Vietnamese', 'l_Yiddish', 'c_Argentina', 'c_Australia',
       'c_Bahamas', 'c_Canada', 'c_China', 'c_CzechRepublic', 'c_Egypt',
       'c_France', 'c_Germany', 'c_India', 'c_Ireland', 'c_Italy', 'c_Japan',
       'c_Netherlands', 'c_NewZealand', 'c_SouthAfrica', 'c_SouthKorea',
       'c_Spain', 'c_UK', 'c_USA'],
      dtype

In [ ]:
df_join.to_csv("metadata.csv",index=False)